In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import hvplot.pandas

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ChrisClass")

from scipy.stats import linregress
from citipy import citipy
from api_keys import bea_key

In [2]:
#Upload landfall and damage CSVs to notebook
landfall_df = pd.read_csv("Resources/landfall.csv")
damage_df = pd.read_csv("Resources/costliness.csv")

#Merge the two data frames into one
merged_df = pd.merge(landfall_df,damage_df,on = ['Season','Name'],how='outer')

#Drop rows with empty columns
merged_df = merged_df.dropna()

#Rename Season columns to Year
merged_df = merged_df.rename(columns={'Season':'Year'})

#Show dataframe preview
merged_df.head()


,#,Date,Year,Time,Latitude,Longitude,Max Winds (kt),SS HWS,RMW nm,Central Prssure (mb),States Affected,Name,Category,Adjusted Costs
1,3.0,8/18/83,1983,0700Z,29.1,-95.1,100.0,3.0,10,962,CTX3,Alicia,3.0,$9.4B
5,5.0,9/2/85,1985,1300Z,30.4,-89.2,100.0,3.0,10,959,"AL3,MS3,AFL3",Elena,3.0,$3.8B
8,12.0,10/29/85,1985,1100Z,29.5,-91.4,75.0,1.0,15,971,LA1,Juan,1.0,$4.3B
15,11.0,9/22/89,1989,0400Z,32.8,-79.8,120.0,4.0,20,934,"SC4,INC1",Hugo,4.0,$22.7B
17,3.0,8/19/91,1991,1720Z,41.2,-71.6,90.0,2.0,25,962,"RI2,MA2,NY2,CT2",Bob,2.0,$3.5B


In [6]:
#Create lists to store states, counties, and coordinates.
counties = []
states = []
lat_lngs = []

#Convert Lat and Long to a string
merged_df['Longitude'] = merged_df['Longitude'].astype(str)
merged_df['Latitude'] = merged_df['Latitude'].astype(str)

#Add lattitude and longitude values to list
lat_lngs = merged_df[['Latitude','Longitude']].values.tolist()

#Loop through lat_lng list to pull county and state names with geopy  
for lat_lng in lat_lngs:
    try:
        county = geolocator.reverse(lat_lng[0]+","+lat_lng[1])
        county = county.raw['address']['county']
        counties.append(county)
        state = geolocator.reverse(lat_lng[0]+","+lat_lng[1])
        state = state.raw['address']['state']
        states.append(state)
    #To handle errors if data isn't found
    except:
        counties.append('')
        states.append('')

#Add county and state names to dataframe
merged_df['County'] = counties
merged_df['State'] = states
merged_df['County'] = merged_df['County'].str.replace(' County','')
merged_df['County'] = merged_df['County'].str.replace(' Parish','')

#Dictionary to store State names and abb.
state_to_abbrev = {"Alabama": "AL","Alaska": "AK","Arizona": "AZ","Arkansas": "AR","California": "CA",
    "Colorado": "CO","Connecticut": "CT","Delaware": "DE","Florida": "FL","Georgia": "GA",
    "Hawaii": "HI","Idaho": "ID","Illinois": "IL","Indiana": "IN","Iowa": "IA",
    "Kansas": "KS","Kentucky": "KY","Louisiana": "LA","Maine": "ME","Maryland": "MD",
    "Massachusetts": "MA","Michigan": "MI","Minnesota": "MN","Mississippi": "MS","Missouri": "MO",
    "Montana": "MT","Nebraska": "NE","Nevada": "NV","New Hampshire": "NH","New Jersey": "NJ",
    "New Mexico": "NM","New York": "NY","North Carolina": "NC","North Dakota": "ND","Ohio": "OH",
    "Oklahoma": "OK","Oregon": "OR","Pennsylvania": "PA","Rhode Island": "RI","South Carolina": "SC",
    "South Dakota": "SD","Tennessee": "TN","Texas": "TX","Utah": "UT","Vermont": "VT",
    "Virginia": "VA","Washington": "WA","West Virginia": "WV","Wisconsin": "WI","Wyoming": "WY","District of Columbia": "DC"}

#Replace State Names with codes
merged_df['State'] = merged_df['State'].replace(state_to_abbrev)
merged_df.head()

,#,Date,Year,Time,Latitude,Longitude,Max Winds (kt),SS HWS,RMW nm,Central Prssure (mb),States Affected,Name,Category,Adjusted Costs,County,State
1,3.0,8/18/83,1983,0700Z,29.1,-95.1,100.0,3.0,10,962,CTX3,Alicia,3.0,$9.4B,Galveston,TX
5,5.0,9/2/85,1985,1300Z,30.4,-89.2,100.0,3.0,10,959,"AL3,MS3,AFL3",Elena,3.0,$3.8B,Harrison,MS
8,12.0,10/29/85,1985,1100Z,29.5,-91.4,75.0,1.0,15,971,LA1,Juan,1.0,$4.3B,St. Mary,LA
15,11.0,9/22/89,1989,0400Z,32.8,-79.8,120.0,4.0,20,934,"SC4,INC1",Hugo,4.0,$22.7B,Charleston,SC
17,3.0,8/19/91,1991,1720Z,41.2,-71.6,90.0,2.0,25,962,"RI2,MA2,NY2,CT2",Bob,2.0,$3.5B,South,RI


In [7]:
#Save new csv to avoid long run time of above code 
merged_df.to_csv('Resources/merged_df.csv',index=False)

In [3]:
#Upload new merged csv to notebook
hurricanes_df = pd.read_csv('Resources/merged_df.csv')
hurricanes_df.head()

,#,Date,Year,Time,Latitude,Longitude,Max Winds (kt),SS HWS,RMW nm,Central Prssure (mb),States Affected,Name,Category,Adjusted Costs,County,State
0,3.0,8/18/83,1983,0700Z,29.1,-95.1,100.0,3.0,10,962,CTX3,Alicia,3.0,$9.4B,Galveston,TX
1,5.0,9/2/85,1985,1300Z,30.4,-89.2,100.0,3.0,10,959,"AL3,MS3,AFL3",Elena,3.0,$3.8B,Harrison,MS
2,12.0,10/29/85,1985,1100Z,29.5,-91.4,75.0,1.0,15,971,LA1,Juan,1.0,$4.3B,St. Mary,LA
3,11.0,9/22/89,1989,0400Z,32.8,-79.8,120.0,4.0,20,934,"SC4,INC1",Hugo,4.0,$22.7B,Charleston,SC
4,3.0,8/19/91,1991,1720Z,41.2,-71.6,90.0,2.0,25,962,"RI2,MA2,NY2,CT2",Bob,2.0,$3.5B,South,RI


In [4]:
#Upload HPI csv to dataframe
hpi_df = pd.read_csv('Resources/hpi_county.csv')
hpi_df['Year'] = hpi_df['Year'].astype(str)
hpi_df['FIPS code'] = hpi_df['FIPS code'].astype(str)
hpi_df['FIPS code'] = hpi_df['FIPS code'].apply(lambda x: x.zfill(5))
hpi_df.replace('.',"",inplace=True)
hpi_df

,State,County,FIPS code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
0,AL,Autauga,01001,1986,,100,95.59,70.96
1,AL,Autauga,01001,1987,-1.94,98.06,93.74,69.59
2,AL,Autauga,01001,1988,2.57,100.58,96.14,71.38
3,AL,Autauga,01001,1989,4.32,104.92,100.29,74.46
4,AL,Autauga,01001,1990,-0.29,104.62,100,74.24
...,...,...,...,...,...,...,...,...
100427,WY,Weston,56045,2019,8.33,223.4,,201.09
100428,WY,Weston,56045,2020,4.36,233.13,,209.86
100429,WY,Weston,56045,2021,4.93,244.62,,220.2
100430,WY,Weston,56045,2022,6.87,261.41,,235.32


In [5]:
base_url = 'https://apps.bea.gov/api/data'
year_range = '1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023'

county_url = f'{base_url}?&UserID={bea_key}&method=GetData&datasetname=Regional&TableName=CAINC1&LineCode=1&Year={year_range}&GeoFips=COUNTY&ResultFormat=json'
county_json = requests.get(county_url).json()

json_df = pd.json_normalize(county_json['BEAAPI']['Results']['Data'])
income_df = json_df[['GeoFips','GeoName','TimePeriod','DataValue']]
income_df = income_df.rename(columns={'GeoFips':'FIPS code','TimePeriod':'Year','GeoName':'County & State','DataValue':'PerCapita Income'})
income_df['Year'] = income_df['Year'].astype(str)
income_df['PerCapita Income'] = income_df['PerCapita Income'].astype(float)
income_df['PerCapita Income'] = income_df['PerCapita Income']/100
income_df

,FIPS code,County & State,Year,PerCapita Income
0,01001,"Autauga, AL",1988,4656.42
1,01001,"Autauga, AL",1991,5750.36
2,01001,"Autauga, AL",2003,12401.94
3,01001,"Autauga, AL",2006,15368.55
4,01001,"Autauga, AL",2017,22903.51
...,...,...,...,...
135015,56045,"Weston, WY",2004,1868.67
135016,56045,"Weston, WY",1985,1071.70
135017,56045,"Weston, WY",1995,1379.35
135018,56045,"Weston, WY",2001,1840.73


In [6]:
#Merge income an hpi datasets
income_hpi_df = pd.merge(hpi_df,income_df,on = ['FIPS code','Year'],how='outer')
income_hpi_df = income_hpi_df.dropna(subset=['HPI','PerCapita Income'])
income_hpi_df

,State,County,FIPS code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base,County & State,PerCapita Income
6,AL,Autauga,01001,1986,,100,95.59,70.96,"Autauga, AL",3906.85
7,AL,Autauga,01001,1987,-1.94,98.06,93.74,69.59,"Autauga, AL",4242.53
8,AL,Autauga,01001,1988,2.57,100.58,96.14,71.38,"Autauga, AL",4656.42
9,AL,Autauga,01001,1989,4.32,104.92,100.29,74.46,"Autauga, AL",5031.59
10,AL,Autauga,01001,1990,-0.29,104.62,100,74.24,"Autauga, AL",5318.09
...,...,...,...,...,...,...,...,...,...,...
143045,WY,Weston,56045,2018,-0.9,206.21,,185.62,"Weston, WY",3102.31
143046,WY,Weston,56045,2019,8.33,223.4,,201.09,"Weston, WY",3160.01
143047,WY,Weston,56045,2020,4.36,233.13,,209.86,"Weston, WY",3399.07
143048,WY,Weston,56045,2021,4.93,244.62,,220.2,"Weston, WY",3365.03


In [7]:
#Create lists to store the year values of the years around a hurricane strike
previous_year = []
year_1 = []
year_2 = []
year_3 = []

#Create lists to store the Annual change values of the years around a hurricane strike
prev_ac = []
year_1_ac = []
year_2_ac = []
year_3_ac = []

#Create lists to store the HPI values of the years around a hurricane strike
prev_hpi = []
year_1_hpi = []
year_2_hpi = []
year_3_hpi = []

#Create lists to store the Median Incomes values of the years around a hurricane strike
prev_mi = []
year_1_mi = []
year_2_mi = []
year_3_mi = []

#Loop through the hpi data and pull the needed year values
for index in income_hpi_df.index:
    try:
        value = income_hpi_df.at[index-1,'Year']
        previous_year.append(value)
    except:
        previous_year.append('')

for index in income_hpi_df.index:
    try:
        value = income_hpi_df.at[index+1,'Year']
        year_1.append(value)
    except:
        year_1.append('')

for index in income_hpi_df.index:
    try:
        value = income_hpi_df.at[index+2,'Year']
        year_2.append(value)
    except:
        year_2.append('')

for index in income_hpi_df.index:
    try:
        value = income_hpi_df.at[index+3,'Year']
        year_3.append(value)
    except:
        year_3.append('')

#Loop through the hpi data and pull the needed Annual Change values
for index in income_hpi_df.index:
    try:
        value = income_hpi_df.at[index-1,'Annual Change (%)']
        prev_ac.append(value)
    except:
        prev_ac.append('')

for index in income_hpi_df.index:
    try:
        value = income_hpi_df.at[index+1,'Annual Change (%)']
        year_1_ac.append(value)
    except:
        year_1_ac.append('')

for index in income_hpi_df.index:
    try:
        value = income_hpi_df.at[index+2,'Annual Change (%)']
        year_2_ac.append(value)
    except:
        year_2_ac.append('')

for index in income_hpi_df.index:
    try:
        value = income_hpi_df.at[index+3,'Annual Change (%)']
        year_3_ac.append(value)
    except:
        year_3_ac.append('')

#Loop through the hpi data and pull the needed HPI values
for index in income_hpi_df.index:
    try:
        value = income_hpi_df.at[index-1,'HPI']
        prev_hpi.append(value)
    except:
        prev_hpi.append('')

for index in income_hpi_df.index:
    try:
        value = income_hpi_df.at[index+1,'HPI']
        year_1_hpi.append(value)
    except:
        year_1_hpi.append('')

for index in income_hpi_df.index:
    try:
        value = income_hpi_df.at[index+2,'HPI']
        year_2_hpi.append(value)
    except:
        year_2_hpi.append('')

for index in income_hpi_df.index:
    try:
        value = income_hpi_df.at[index+3,'HPI']
        year_3_hpi.append(value)
    except:
        year_3_hpi.append('')

#Loop through the hpi data and pull the needed year values
for index in income_hpi_df.index:
    try:
        value = income_hpi_df.at[index-1,'PerCapita Income']
        prev_mi.append(value)
    except:
        prev_mi.append('')

for index in income_hpi_df.index:
    try:
        value = income_hpi_df.at[index+1,'PerCapita Income']
        year_1_mi.append(value)
    except:
        year_1_mi.append('')

for index in income_hpi_df.index:
    try:
        value = income_hpi_df.at[index+2,'PerCapita Income']
        year_2_mi.append(value)
    except:
        year_2_mi.append('')

for index in income_hpi_df.index:
    try:
        value = income_hpi_df.at[index+3,'PerCapita Income']
        year_3_mi.append(value)
    except:
        year_3_mi.append('')

#Create new columns and place the values on the data frame
income_hpi_df['Year -1'] = previous_year
income_hpi_df['Year +1'] = year_1
income_hpi_df['Year +2'] = year_2
income_hpi_df['Year +3'] = year_3
income_hpi_df['Annual Change (%) -1'] = prev_ac
income_hpi_df['Annual Change (%) +1'] = year_1_ac
income_hpi_df['Annual Change (%) +2'] = year_2_ac
income_hpi_df['Annual Change (%) +3'] = year_3_ac
income_hpi_df['HPI -1'] = prev_hpi
income_hpi_df['HPI +1'] = year_1_hpi
income_hpi_df['HPI +2'] = year_2_hpi
income_hpi_df['HPI +3'] = year_3_hpi
income_hpi_df['PerCapita Income -1'] = prev_mi
income_hpi_df['PerCapita Income +1'] = year_1_mi
income_hpi_df['PerCapita Income +2'] = year_2_mi
income_hpi_df['PerCapita Income +3'] = year_3_mi
income_hpi_df.head()


,State,County,FIPS code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base,County & State,PerCapita Income,...,Annual Change (%) +2,Annual Change (%) +3,HPI -1,HPI +1,HPI +2,HPI +3,PerCapita Income -1,PerCapita Income +1,PerCapita Income +2,PerCapita Income +3
6,AL,Autauga,01001,1986,,100,95.59,70.96,"Autauga, AL",3906.85,...,2.57,4.32,,98.06,100.58,104.92,,4242.53,4656.42,5031.59
7,AL,Autauga,01001,1987,-1.94,98.06,93.74,69.59,"Autauga, AL",4242.53,...,4.32,-0.29,100,100.58,104.92,104.62,3906.85,4656.42,5031.59,5318.09
8,AL,Autauga,01001,1988,2.57,100.58,96.14,71.38,"Autauga, AL",4656.42,...,-0.29,2.52,98.06,104.92,104.62,107.25,4242.53,5031.59,5318.09,5750.36
9,AL,Autauga,01001,1989,4.32,104.92,100.29,74.46,"Autauga, AL",5031.59,...,2.52,3.09,100.58,104.62,107.25,110.57,4656.42,5318.09,5750.36,6138.12
10,AL,Autauga,01001,1990,-0.29,104.62,100,74.24,"Autauga, AL",5318.09,...,3.09,3.35,104.92,107.25,110.57,114.28,5031.59,5750.36,6138.12,6537.96


In [8]:
#Save combined hpi and income df to directory
income_hpi_df.to_csv('Output_CSVs/hpi_income_df.csv',index=False)

In [9]:
hpi_income_df_csv = pd.read_csv('Output_CSVs/hpi_income_df.csv')
hpi_income_df_csv['FIPS code'] = hpi_income_df_csv['FIPS code'].astype(str)
hpi_income_df_csv['FIPS code'] = hpi_income_df_csv['FIPS code'].apply(lambda x: x.zfill(5))
hpi_income_df_csv.head()

,State,County,FIPS code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base,County & State,PerCapita Income,...,Annual Change (%) +2,Annual Change (%) +3,HPI -1,HPI +1,HPI +2,HPI +3,PerCapita Income -1,PerCapita Income +1,PerCapita Income +2,PerCapita Income +3
0,AL,Autauga,01001,1986,NaN,100.00,95.59,70.96,"Autauga, AL",3906.85,...,2.57,4.32,NaN,98.06,100.58,104.92,NaN,4242.53,4656.42,5031.59
1,AL,Autauga,01001,1987,-1.94,98.06,93.74,69.59,"Autauga, AL",4242.53,...,4.32,-0.29,100.00,100.58,104.92,104.62,3906.85,4656.42,5031.59,5318.09
2,AL,Autauga,01001,1988,2.57,100.58,96.14,71.38,"Autauga, AL",4656.42,...,-0.29,2.52,98.06,104.92,104.62,107.25,4242.53,5031.59,5318.09,5750.36
3,AL,Autauga,01001,1989,4.32,104.92,100.29,74.46,"Autauga, AL",5031.59,...,2.52,3.09,100.58,104.62,107.25,110.57,4656.42,5318.09,5750.36,6138.12
4,AL,Autauga,01001,1990,-0.29,104.62,100.00,74.24,"Autauga, AL",5318.09,...,3.09,3.35,104.92,107.25,110.57,114.28,5031.59,5750.36,6138.12,6537.96


In [10]:
#Create an aggregate table to determine mean and median by year by state and by nation
by_nation = hpi_income_df_csv.groupby(['Year'])
change_by_nation = by_nation['Annual Change (%)'].agg(['mean'])
change_by_nation.head()


,mean
Year,
1980,7.590396
1981,4.567931
1982,0.715065
1983,7.350132
1984,3.630926


In [13]:
prev_mean_national = []
mean_national_1 = []
mean_national_2 = []
mean_national_3 = []

for index in change_by_nation.index:
    try:
        value = change_by_nation.at[index-1,'mean']
        prev_mean_national.append(value)
    except:
        prev_mean_national.append('')

for index in change_by_nation.index:
    try:
        value = change_by_nation.at[index+1,'mean']
        mean_national_1.append(value)
    except:
        mean_national_1.append('')

for index in change_by_nation.index:
    try:
        value = change_by_nation.at[index+2,'mean']
        mean_national_2.append(value)
    except:
        mean_national_2.append('')

for index in change_by_nation.index:
    try:
        value = change_by_nation.at[index+3,'mean']
        mean_national_3.append(value)
    except:
        mean_national_3.append('')

change_by_nation['National AC Mean -1'] = prev_mean_national
change_by_nation['National AC Mean +1'] = mean_national_1
change_by_nation['National AC Mean +2'] = mean_national_2
change_by_nation['National AC Mean +3'] = mean_national_3
change_by_nation.head()

,mean,National AC Mean -1,National AC Mean +1,National AC Mean +2,National AC Mean +3
Year,,,,,
1980,7.590396,,4.567931,0.715065,7.350132
1981,4.567931,7.590396,0.715065,7.350132,3.630926
1982,0.715065,4.567931,7.350132,3.630926,3.304653
1983,7.350132,0.715065,3.630926,3.304653,4.337647
1984,3.630926,7.350132,3.304653,4.337647,3.272305


In [14]:
by_state = hpi_income_df_csv.groupby(['State','Year'])
change_by_state = by_state['Annual Change (%)'].agg(['mean'])
change_by_state = change_by_state.reset_index(level=1)
change_by_state.head()

,Year,mean
State,,
AK,1980,4.775
AK,1981,15.595
AK,1982,12.500
AK,1983,8.110
AK,1984,11.970


In [15]:
change_by_state['State AC Mean -1'] = change_by_state.groupby('State')['mean'].shift(-1)
change_by_state['State AC Mean +1'] = change_by_state.groupby('State')['mean'].shift(1)
change_by_state['State AC Mean +2'] = change_by_state.groupby('State')['mean'].shift(2)
change_by_state['State AC Mean +3'] = change_by_state.groupby('State')['mean'].shift(3)
change_by_state

,Year,mean,State AC Mean -1,State AC Mean +1,State AC Mean +2,State AC Mean +3
State,,,,,,
AK,1980,4.775000,15.595000,NaN,NaN,NaN
AK,1981,15.595000,12.500000,4.775000,NaN,NaN
AK,1982,12.500000,8.110000,15.595000,4.775000,NaN
AK,1983,8.110000,11.970000,12.500000,15.595000,4.775000
AK,1984,11.970000,-5.578000,8.110000,12.500000,15.595000
...,...,...,...,...,...,...
WY,2018,3.183182,3.885000,1.372273,3.335909,2.755455
WY,2019,3.885000,3.487727,3.183182,1.372273,3.335909
WY,2020,3.487727,9.633636,3.885000,3.183182,1.372273


In [16]:
#Add the state and national data to the HPI dataframe 
merged_df = pd.merge(hpi_income_df_csv,change_by_nation,on = ['Year'],how='outer')
clean_new_hpi_income_df = pd.merge(merged_df,change_by_state,on=['Year','State'],how='outer')
clean_new_hpi_income_df = clean_new_hpi_income_df.rename(columns={'mean_x':'National AC Mean','median_x':'National AC Median',
                                          'mean_y':'State AC Mean','median_y':'State AC Median'})
#Display dataframe
clean_new_hpi_income_df


,State,County,FIPS code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base,County & State,PerCapita Income,...,National AC Mean,National AC Mean -1,National AC Mean +1,National AC Mean +2,National AC Mean +3,State AC Mean,State AC Mean -1,State AC Mean +1,State AC Mean +2,State AC Mean +3
0,AK,Anchorage,02020,1980,0.39,148.12,83.03,57.49,"Anchorage Municipality, AK",29726.12,...,7.590396,,4.567931,0.715065,7.350132,4.775000,15.595,NaN,NaN,NaN
1,AK,Fairbanks North Star,02090,1980,NaN,NaN,NaN,NaN,"Fairbanks North Star Borough, AK",8266.35,...,7.590396,,4.567931,0.715065,7.350132,4.775000,15.595,NaN,NaN,NaN
2,AK,Juneau,02110,1980,9.16,109.16,78.71,46.68,"Juneau City and Borough, AK",4015.59,...,7.590396,,4.567931,0.715065,7.350132,4.775000,15.595,NaN,NaN,NaN
3,AL,Baldwin,01003,1980,18.11,124.78,86.25,52.00,"Baldwin, AL",6159.85,...,7.590396,,4.567931,0.715065,7.350132,6.905833,2.655,NaN,NaN,NaN
4,AL,Calhoun,01015,1980,3.95,131.07,79.42,60.18,"Calhoun, AL",9730.80,...,7.590396,,4.567931,0.715065,7.350132,6.905833,2.655,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92396,WY,Sweetwater,56037,2022,9.29,632.15,322.50,222.46,"Sweetwater, WY",24134.86,...,15.900497,10.929004,,,,15.822727,NaN,9.633636,3.487727,3.885
92397,WY,Teton,56039,2022,4.85,1179.97,829.10,328.87,"Teton, WY",94557.80,...,15.900497,10.929004,,,,15.822727,NaN,9.633636,3.487727,3.885
92398,WY,Uinta,56041,2022,19.03,351.46,507.17,289.71,"Uinta, WY",9273.86,...,15.900497,10.929004,,,,15.822727,NaN,9.633636,3.487727,3.885
92399,WY,Washakie,56043,2022,14.09,332.95,NaN,225.37,"Washakie, WY",4267.69,...,15.900497,10.929004,,,,15.822727,NaN,9.633636,3.487727,3.885


In [17]:
#Merge HPI data with hurricane data
hurricane_hpi_income_df = pd.merge(hurricanes_df,clean_new_hpi_income_df,on=['Year','County','State'],how='inner')
hurricane_hpi_income_df.head()

,#,Date,Year,Time,Latitude,Longitude,Max Winds (kt),SS HWS,RMW nm,Central Prssure (mb),...,National AC Mean,National AC Mean -1,National AC Mean +1,National AC Mean +2,National AC Mean +3,State AC Mean,State AC Mean -1,State AC Mean +1,State AC Mean +2,State AC Mean +3
0,3.0,8/18/83,1983,0700Z,29.1,-95.1,100.0,3.0,10,962,...,7.350132,0.715065,3.630926,3.304653,4.337647,9.237719,1.364219,3.585926,5.788889,11.250678
1,5.0,9/2/85,1985,1300Z,30.4,-89.2,100.0,3.0,10,959,...,3.304653,3.630926,4.337647,3.272305,3.583211,6.701818,0.500000,2.293636,1.566667,3.806667
2,11.0,9/22/89,1989,0400Z,32.8,-79.8,120.0,4.0,20,934,...,4.637415,3.583211,3.369176,2.911246,2.990405,3.666923,3.289259,3.210769,4.107692,5.356316
3,4.0,8/24/92,1992,0905Z,25.5,-80.3,145.0,5.0,10,922,...,2.990405,2.911246,3.453711,4.485092,5.729116,2.358750,2.001818,1.079000,2.099500,4.601250
4,17.0,10/4/95,1995,2200Z,30.3,-87.1,100.0,3.0,50,942,...,5.729116,4.485092,4.005931,4.197426,3.915739,3.318936,3.638800,0.763191,2.001818,2.358750


In [19]:
#Make a new dataframe with only relevant columns
clean_hurricane_hpi_income_df = hurricane_hpi_income_df[['Name','Max Winds (kt)','Adjusted Costs','FIPS code','County','State',
                        'Year -1','Year','Year +1','Year +2',
                        'Annual Change (%) -1','Annual Change (%)','Annual Change (%) +1','Annual Change (%) +2',
                        'HPI -1','HPI','HPI +1','HPI +2',
                        'National AC Mean -1','National AC Mean','National AC Mean +1','National AC Mean +2',
                        'State AC Mean -1','State AC Mean','State AC Mean +1','State AC Mean +2',
                        'PerCapita Income -1','PerCapita Income','PerCapita Income +1','PerCapita Income +2']]
clean_hurricane_hpi_income_df = clean_hurricane_hpi_income_df[(clean_hurricane_hpi_income_df['Year']<2022) & (clean_hurricane_hpi_income_df['Year']>1980)]
clean_hurricane_hpi_income_df

,Name,Max Winds (kt),Adjusted Costs,FIPS code,County,State,Year -1,Year,Year +1,Year +2,...,National AC Mean +1,National AC Mean +2,State AC Mean -1,State AC Mean,State AC Mean +1,State AC Mean +2,PerCapita Income -1,PerCapita Income,PerCapita Income +1,PerCapita Income +2
0,Alicia,100.0,$9.4B,48167,Galveston,TX,1982.0,1983,1984.0,1985.0,...,3.630926,3.304653,1.364219,9.237719,3.585926,5.788889,26082.76,27298.81,29340.55,30531.12
1,Elena,100.0,$3.8B,28047,Harrison,MS,1984.0,1985,1986.0,1987.0,...,4.337647,3.272305,0.500000,6.701818,2.293636,1.566667,18440.68,19473.11,20579.73,21226.86
2,Hugo,120.0,$22.7B,45019,Charleston,SC,1988.0,1989,1990.0,1991.0,...,3.369176,2.911246,3.289259,3.666923,3.210769,4.107692,45928.31,49382.78,53736.43,56151.47
3,Andrew,145.0,$60.2B,12086,Miami-Dade,FL,1991.0,1992,1993.0,1994.0,...,3.453711,4.485092,2.001818,2.358750,1.079000,2.099500,370009.65,385955.82,405013.48,420738.53
4,Opal,100.0,$9.6B,12033,Escambia,FL,1994.0,1995,1996.0,1997.0,...,4.005931,4.197426,3.638800,3.318936,0.763191,2.001818,51871.65,54831.40,58648.14,61972.58
5,Fran,100.0,$10.0B,37019,Brunswick,NC,1995.0,1996,1997.0,1998.0,...,4.197426,3.915739,4.788235,3.822892,5.911772,3.315128,10811.48,11898.70,13194.24,14579.64
6,Georges,90.0,$11.5B,12087,Monroe,FL,1997.0,1998,1999.0,2000.0,...,3.739902,5.013175,3.443036,4.533208,3.337358,3.638800,25659.03,28208.84,28535.08,31506.09
7,Georges,90.0,$11.5B,28047,Harrison,MS,1997.0,1998,1999.0,2000.0,...,3.739902,5.013175,5.001818,4.278913,4.939348,3.377500,39177.91,42621.37,45767.25,48670.64
8,Floyd,90.0,$12.1B,37019,Brunswick,NC,1998.0,1999,2000.0,2001.0,...,5.013175,5.292247,5.468351,3.728211,4.212069,4.788235,14579.64,15674.26,16879.83,18605.27
9,Isabel,90.0,$9.3B,37031,Carteret,NC,2002.0,2003,2004.0,2005.0,...,6.075588,8.569651,5.103434,2.179596,3.062525,5.113980,16838.54,17994.53,19375.86,20605.54


In [21]:
#Save hurricanes df to directory
hurricanes_df.to_csv('Output_CSVs/hurricanes_df.csv',index=False)

#Save Merged df to directory
clean_new_hpi_income_df.to_csv('Output_CSVs/clean_new_hpi_income.csv',index=False)

#Save clean df as CSV to directory
clean_hurricane_hpi_income_df.to_csv('Output_CSVs/clean_hurricane_hpi_income_data.csv',index=False)



hurricanes_df = Total Data for hurricane strikes in the US

hpi_df = Total HPI data with new columns for every county

clean_hurricane_hpi_df = Comined previous two data frames. All hurricane strikes with HPI data.